In [2]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch, helpers
import requests
from IPython.display import JSON
import numpy as np
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
import pickle
from IPython.display import clear_output, display
from colavsim import colav_similarity, parse_doi, parse_string

In [3]:
es = Elasticsearch()

In [3]:
openalex = list(MongoClient()["openalexco"]["works"].find({'doi': {"$ne": None}},{"_id":0}))

In [19]:
for i in range(len(openalex)):
    openalex[i]['title'] = parse_string(openalex[i]['title'])

In [20]:
len(openalex)

203863

In [53]:
# settings = {
#   "settings": {
#     "index": {
#       "similarity": {
#         "default": {
#           "type": "DFR",
#           "basic_model": "g",
#           "after_effect": "b",
#           "normalization": "h2",
#           "normalization.h2.c": "600.0"  # Cambia el valor según tus necesidades
#         }
#       }
#     }
#   }
# }
# settings = {
#   "settings": {
#     "index": {
#       "similarity": {
#         "default": {
#           "type": "BM25",
#           "b": 0.80,   # Ajusta el parámetro "b" de BM25
#           "k1": 2.1   # Cambia el valor según tus necesidades
#         }
#       }
#     }
#   }
# }
settings={}

In [54]:
for i in range(len(openalex)):
    openalex[i]['title'] = parse_string(openalex[i]['title'])

In [4]:
es_index = "oa"
index_size = 5000
recreate = True
if recreate:
    # Obtener una lista de todos los índices
    indices = es.indices.get_alias("*").keys()

    # Eliminar cada índice
    for index in indices:
        es.indices.delete(index=index)
    #for i in range(int(len(openalex)/index_size)):
    #    if es.indices.exists(index=es_index):
    #        es.indices.delete(index=es_index)


/tmp/ipykernel_105520/1927439833.py:6: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  indices = es.indices.get_alias("*").keys()


In [56]:
if recreate:
    
    request_timeout = 60
    bulk_size = 100
    es_entries = []
    counter = 0
    index_n = 0
    for i in openalex:
        if counter%index_size == 0:
            index_n=int(index_n)+1
            if not es.indices.exists(index=es_index+str(index_n)):
                es.indices.create(index=es_index+str(index_n),body=settings)
#                es.indices.create(index=es_index+str(index_n),document=settings)

                #es.indices.put_settings(index=es_index+str(index_n), body=settings)
        entry = {"_index": es_index+str(index_n),
                 "_id": counter,
                 "_source": i}
        es_entries.append(entry)
        if len(es_entries) == bulk_size:
            helpers.bulk(es, es_entries, refresh=True,request_timeout=request_timeout)
            es_entries = []
        counter +=1

/tmp/ipykernel_239436/3024032966.py:12: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=es_index+str(index_n),body=settings)
